# **Modelo Solución: Clasificación de Rendimiento Estudiantil**
---

### **Modelo a Usar: LightGBM**

Para este caso, se ha seleccionado el modelo **LightGBM (Light Gradient Boosting Machine)**. Conocido por su eficiencia y velocidad en el entrenamiento de modelos de aprendizaje automático basados en árboles de decisión

### **Fase 1: Configuración del Entorno**

En esta sección, cargamos todas las librerías necesarias para el proyecto.

In [2]:
!pip install lightgbm -q --upgrade

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc
from datetime import datetime


### **Fase 2: Preparación de Datos**

Carga de datos, limpieza, transformación de variables y creación de nuevas características (Feature Engineering) para mejorar el modelo.

In [3]:
def preparar_datos(ruta_archivo):
    """Carga y aplica todas las transformaciones a un archivo de datos."""
    df = pd.read_csv(ruta_archivo)

    # --- Limpieza de Datos ---
    # Eliminar columnas con un solo valor único, ya que no aportan información.
    for col in df.columns:
        if df[col].nunique(dropna=False) <= 1:
            df.drop(columns=[col], inplace=True)

    # Imputar valores nulos en columnas de texto con el valor más frecuente (moda).
    for col in df.select_dtypes(include=['object']).columns:
        if col != 'RENDIMIENTO_GLOBAL': # No tocar la variable objetivo
            df[col].fillna(df[col].mode()[0], inplace=True)

    # --- Transformación de Variables (Encoding) ---
    # Mapeo de variables binarias (Si/No).
    si_no_map = {'Si': 1, 'No': 0, 'S': 1, 'N': 0}
    for col in df.select_dtypes(include=['object']).columns:
        if set(df[col].dropna().unique()).issubset(si_no_map.keys()):
            df[col] = df[col].map(si_no_map).astype('Int8')

    # Mapeo de variables ordinales (educación, ingresos, etc.).
    edu_map = {'Ninguno': 0, 'No sabe': 0, 'Primaria incompleta': 1, 'Primaria completa': 2, 'Secundaria (Bachillerato) incompleta': 3, 'Secundaria (Bachillerato) completa': 4, 'Técnica o tecnológica incompleta': 5, 'Técnica o tecnológica completa': 6, 'Postgrado': 7}
    if 'FAMI_EDUCACIONPADRE' in df.columns: df['FAMI_EDUCACIONPADRE'] = df['FAMI_EDUCACIONPADRE'].map(edu_map)
    if 'FAMI_EDUCACIONMADRE' in df.columns: df['FAMI_EDUCACIONMADRE'] = df['FAMI_EDUCACIONMADRE'].map(edu_map)
    if 'ESTU_VALORMATRICULAUNIVERSIDAD' in df.columns: df['ESTU_VALORMATRICULAUNIVERSIDAD'] = df['ESTU_VALORMATRICULAUNIVERSIDAD'].map({'No pagó matrícula': 0, 'Menos de 500 mil': 1, 'Entre 500 mil y menos de 1 millón': 2, 'Entre 1 millón y menos de 2.5 millones': 3, 'Entre 2.5 millones y menos de 4 millones': 4, 'Entre 4 millones y menos de 5.5 millones': 5, 'Entre 5.5 millones y menos de 7 millones': 6, 'Más de 7 millones': 7})
    if 'ESTU_HORASSEMANATRABAJA' in df.columns: df['ESTU_HORASSEMANATRABAJA'] = df['ESTU_HORASSEMANATRABAJA'].map({'0': 0, 'Menos de 10 horas': 1, 'Entre 11 y 20 horas': 2, 'Entre 21 y 30 horas': 3, 'Más de 30 horas': 4})
    if 'FAMI_ESTRATOVIVIENDA' in df.columns: df['FAMI_ESTRATOVIVIENDA'] = df['FAMI_ESTRATOVIVIENDA'].str.replace('Estrato ', '').str.replace('Sin Estrato', '0').astype(np.int8)

    # --- Manejo de Categóricas para LightGBM ---
    # Convertir a tipo 'category' para un manejo eficiente en memoria.
    for col in ['ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO']:
        if col in df.columns: df[col] = df[col].astype('category')

    return df


In [4]:
# --- Ejecución de la Preparación ---
datos_entrenamiento = preparar_datos('train.csv')
datos_prueba = preparar_datos('test.csv')

# Separar características (X) y variable objetivo (y)
variable_objetivo = datos_entrenamiento['RENDIMIENTO_GLOBAL']
ids_prueba = datos_prueba['ID']

X = datos_entrenamiento.drop(columns=['RENDIMIENTO_GLOBAL', 'ID'], errors='ignore')
X_submission = datos_prueba.drop(columns=['ID'], errors='ignore')

# Codificar la variable objetivo a números
codificador_etiquetas = LabelEncoder()
y = codificador_etiquetas.fit_transform(variable_objetivo)

del datos_entrenamiento, datos_prueba, variable_objetivo; gc.collect()

/tmp/ipython-input-3-4043954574.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].mode()[0], inplace=True)
/tmp/ipython-input-3-4043954574.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

63

### **Fase 3: Entrenamiento y Evaluación del Modelo**

Dividimos los datos en un conjunto de entrenamiento y uno de validación. Entrenamos el modelo en el primero y lo evaluamos en el segundo para tener una estimación realista de su rendimiento.

Utilizamos una técnica clave llamada **Early Stopping**: el entrenamiento se detiene automáticamente cuando el rendimiento en el set de validación deja de mejorar, lo que previene el sobreajuste y acelera el proceso.

In [5]:
# Dividir los datos: 80% para entrenar, 20% para validar
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Identificar las columnas categóricas para pasárselas al modelo
columnas_categoricas = [col for col in X.columns if X[col].dtype.name == 'category']

# Definir el modelo LightGBM con los parámetros
modelo_lgbm = lgb.LGBMClassifier(
    objective='multiclass',
    num_class=len(codificador_etiquetas.classes_),
    metric='multi_logloss',
    n_estimators=2000,      # Un número alto; Early Stopping encontrará el óptimo
    learning_rate=0.03,     # Controla la velocidad de aprendizaje
    num_leaves=31,          # Número de hojas por árbol
    n_jobs=-1,              # Usar todos los núcleos del procesador
    random_state=42         # Para resultados reproducibles
)

modelo_lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='multi_logloss',
    callbacks=[lgb.early_stopping(100, verbose=True)] # Detener si no mejora en 100 rondas
)

predicciones_validacion = modelo_lgbm.predict(X_val)
accuracy = accuracy_score(y_val, predicciones_validacion)

print("--- RESULTADO ---")
print(f"   Accuracy: {accuracy:.5f}")

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.053758 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1620
[LightGBM] [Info] Number of data points in the train set: 554000, number of used features: 19
[LightGBM] [Info] Start training from score -1.371993
[LightGBM] [Info] Start training from score -1.387089
[LightGBM] [Info] Start training from score -1.395033
[LightGBM] [Info] Start training from score -1.391216
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[469]	valid_0's multi_logloss: 1.19242
--- RESULTADO ---
   Accuracy: 0.43903


### **Fase 4: Generación del Archivo de Submission**

Una vez validado el modelo, el paso final es re-entrenarlo utilizando el **100% de los datos de entrenamiento** y el número óptimo de iteraciones encontrado por Early Stopping. Esto asegura que el modelo final sea lo más robusto posible.

Finalmente, se generan las predicciones para los datos de prueba y se guardan en un archivo `.csv` con un nombre único para garantizar la trazabilidad.

In [6]:
iteraciones_optimas = modelo_lgbm.best_iteration_
if iteraciones_optimas == 0: iteraciones_optimas = 500 # Salvaguarda por si el entrenamiento es muy corto

# Definir el modelo final con los parámetros exactos
modelo_final = lgb.LGBMClassifier(
    objective='multiclass', num_class=len(codificador_etiquetas.classes_),
    n_estimators=iteraciones_optimas,
    learning_rate=0.03, num_leaves=31,
    n_jobs=-1, random_state=42
)

# Entrenar con el 100% de los datos
modelo_final.fit(X, y, categorical_feature=columnas_categoricas)

print("\nGenerando predicciones para el archivo de submission...")
predicciones_finales_codificadas = modelo_final.predict(X_submission)
predicciones_finales = codificador_etiquetas.inverse_transform(predicciones_finales_codificadas)

# Nombre de archivo para la submission
nombre_archivo = f'submission_final.csv'

df_submission = pd.DataFrame({'ID': ids_prueba, 'RENDIMIENTO_GLOBAL': predicciones_finales})
df_submission.to_csv(nombre_archivo, index=False)

print(f"\nEl archivo '{nombre_archivo}' ha sido creado.")
print(df_submission.head())

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.334518 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1615
[LightGBM] [Info] Number of data points in the train set: 692500, number of used features: 19
[LightGBM] [Info] Start training from score -1.371991
[LightGBM] [Info] Start training from score -1.387092
[LightGBM] [Info] Start training from score -1.395031
[LightGBM] [Info] Start training from score -1.391216

Generando predicciones para el archivo de submission...

El archivo 'submission_final.csv' ha sido creado.
       ID RENDIMIENTO_GLOBAL
0  550236               bajo
1   98545         medio-alto
2  4